# Imports

In [ ]:
from IPython.display import display, Image
import os
import time
import random
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
import locale

# Preparing Data

In [ ]:
# Set home path to where the 'Dataset' folder resides
HOME = 'set_home_path_here'

dest_folder=f'{HOME}/Dataset'

In [ ]:
# This function loads all file names from a folder in a list.
def load_filenames(folder):
  filenames = []
  for filename in os.listdir(folder):
    filenames.append(filename)
    #filenames.append(os.path.join(folder,filename))
  return filenames

# Counting Ground Truth Instances

In [ ]:
def count_instances(labels,instances):
  for l in range(len(instances)):
    instances[l]=labels.count(str(l))

instances_train=[0,0,0,0,0,0,0,0,0,0]
train_filenames=load_filenames(f'{dest_folder}/train/labels')
gt_labels_train=[]
for i in train_filenames:
  f=open(f'{dest_folder}/train/labels/{i}')
  for j in f:
    line=gt_labels_train.append(j.split()[0])
count_instances(gt_labels_train,instances_train)
print('Train data statistics:')
for i in range(10):
  print(f'Instances of class {i}',instances_train[i])
print('Total instances in train data:',sum(instances_train))
print('Total images in train data:',len(train_filenames))

instances_valid=[0,0,0,0,0,0,0,0,0,0]
valid_filenames=load_filenames(f'{dest_folder}/valid/labels')
gt_labels_valid=[]
for i in valid_filenames:
  f=open(f'{dest_folder}/valid/labels/{i}')
  for j in f:
    line=gt_labels_valid.append(j.split()[0])
count_instances(gt_labels_valid,instances_valid)
print('\nValid data statistics:')
for i in range(10):
  print(f'Instances of class {i}',instances_valid[i])
print('Total instances in valid data:',sum(instances_valid))
print('Total images in valid data:',len(valid_filenames))

instances_test=[0,0,0,0,0,0,0,0,0,0]
test_filenames=load_filenames(f'{dest_folder}/test/labels')
gt_labels_test=[]
for i in test_filenames:
  f=open(f'{dest_folder}/test/labels/{i}')
  for j in f:
    line=gt_labels_test.append(j.split()[0])
count_instances(gt_labels_test,instances_test)
print('\nTest data statistics:')
for i in range(10):
  print(f'Instances of class {i}',instances_test[i])
print('Total instances in test data:',sum(instances_test))
print('Total images in test data:',len(test_filenames))

print('\nTotal statistics')
for i in range(10):
  print(f'Instances of class {i}',instances_train[i]+instances_valid[i]+instances_test[i])
print('Total instances in total data:',sum(instances_train)+sum(instances_valid)+sum(instances_test))
print('Total images in total data:',len(train_filenames)+len(valid_filenames)+len(test_filenames))

Train data statistics:
Instances of class 0 492
Instances of class 1 231
Instances of class 2 54
Instances of class 3 66
Instances of class 4 290
Instances of class 5 42
Instances of class 6 104
Instances of class 7 99
Instances of class 8 344
Instances of class 9 11
Total instances in train data: 1733
Total images in train data: 498

Valid data statistics:
Instances of class 0 51
Instances of class 1 7
Instances of class 2 6
Instances of class 3 5
Instances of class 4 27
Instances of class 5 5
Instances of class 6 9
Instances of class 7 12
Instances of class 8 42
Instances of class 9 2
Total instances in valid data: 166
Total images in valid data: 62

Test data statistics:
Instances of class 0 39
Instances of class 1 17
Instances of class 2 3
Instances of class 3 1
Instances of class 4 36
Instances of class 5 6
Instances of class 6 9
Instances of class 7 8
Instances of class 8 43
Instances of class 9 2
Total instances in test data: 164
Total images in test data: 62

Total statistics
I

# Installing and Importing YOLOv8

In [ ]:
# plot and utils do not work in the latest version, installing a bit older version
!pip install ultralytics==8.0.145

# Importing YOLO
import ultralytics
# The ultralytics package has the YOLO class, used to create neural network models.
from ultralytics import YOLO

# YOLOv8 version checking
ultralytics.checks()

Ultralytics YOLOv8.0.145 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 31.8/78.2 GB disk)


# Selecting Model, Epochs and Image Size

In [ ]:
# List of models and epochs
models=['yolov8n.pt','yolov8s.pt','yolov8m.pt','yolov8l.pt','yolov8x.pt']
epochs=[50,100,150,200,250,300,350,400,450,500]

# Selecting models, epochs, image size and batch size
model_nr=0
epochs_nr=0
image_size=2560
batch_size=16

# Model Training

In [ ]:
# Loading the selected model
model_train = YOLO("{HOME}/runs/detect/train6/weights/last.pt")

# Training the model
#%%time
t1=time.time() # returns time in seconds
model_train.train(data="data.yaml", epochs=50, imgsz=1408, batch=batch_size, device=0, cache=True, workers=0, fliplr=0.0, mosaic=0.0)
t2=time.time()
elapsed_time=round(t2-t1,2)

# Saving model info and timing info to a file
f=open('{HOME}/runs/model{model_nr}{epochs_nr}/model_info.txt','a+')
f.write(f'Model number: {model_nr}{epochs_nr}\n')
f.write(f'Model: {models[model_nr]}\n')
f.write(f'Epochs: {epochs[epochs_nr]}\n')
f.write (f'Model Info: {model.info()}\n')
f.write(f'Wall time: {elapsed_time} sec\n\n')
f.write(f'{"*"*100}\n\n')
f.close()

# Inferring Kennedy Classification of Partially Edentulous Arches from the Detected Bounding Boxes

## Class names
0. __Broken_Root:__ Broken root

1. __PCT:__ Periodontally compromised tooth

2. __Free_R_Max:__ Edentulous area with one free end on right side of maxillary arch

3. __Free_L_Max:__ Edentulous area with one free end on left side of maxillary arch

4. __Not_Free_Max:__ Edentulous area in maxillary arch with teeth on both sides but not crossing the center

5. __Not_Free_Center_Max:__ Edentulous area in maxillary arch with teeth on both sides and crossing the center

6. __Free_R_Mand:__ Edentulous area with one free end on right side of mandibular arch

7. __Free_L_Mand:__ Edentulous area with one free end on left side of mandibular arch

8. __Not_Free_Mand:__ Edentulous area in mandibular arch with teeth on both sides but not crossing the center

9. __Not_Free_Center_Mand:__ Edentulous area in mandibular arch with teeth on both sides and crossing the center

In [ ]:
##Class names
class_name=['Broken_Root','PCT','Free_R_Max','Free_L_Max','Not_Free_Max','Not_Free_Center_Max','Free_R_Mand','Free_L_Mand','Not_Free_Mand','Not_Free_Center_Mand']


# Diagnosing the issues from class labels
def diagnosis(all_detected_classes):
  diagnosis=[]

  # Counting broken roots
  total_broken_roots=all_detected_classes.count('Broken_Root')
  if total_broken_roots==0:
    diagnosis.append('No broken root detected.')
  elif total_broken_roots==1:
    diagnosis.append('A single broken root detected.')
  else:
    diagnosis.append(f'{total_broken_roots} broken roots detected.')

  # Counting periodontally compromised teeth
  total_PCT=all_detected_classes.count('PCT')
  if total_PCT==0:
    diagnosis.append('No periodontally compromised tooth detected.')
  elif total_PCT==1:
    diagnosis.append('A single periodontally compromised tooth detected.')
  else:
    diagnosis.append(f'{total_PCT} periodontally compromised teeth detected.')

  # Detecting Kennedy Classification
  if 'Free_R_Max' in all_detected_classes and 'Free_L_Max' in all_detected_classes:
    diagnosis.append('Bilateral edentulous areas located in the maxillary arch. Kennedy classification: Class I' )
  else:
    if 'Free_L_Max' in all_detected_classes:
          diagnosis.append('A unilateral edentulous area located on the left side of the maxillary arch. Kennedy classification: Class II')
    elif 'Free_R_Max' in all_detected_classes:
          diagnosis.append('A unilateral edentulous area located on the right side of the maxillary arch. Kennedy classification: Class II')
  if 'Free_R_Mand' in all_detected_classes and 'Free_L_Mand' in all_detected_classes:
    diagnosis.append('Bilateral edentulous areas located in the mandibular arch. Kennedy classification: Class I' )
  else:
    if 'Free_L_Mand' in all_detected_classes:
          diagnosis.append('A unilateral edentulous area located on the left side of the mandibular arch. Kennedy classification: Class II')
    elif 'Free_R_Mand' in all_detected_classes:
          diagnosis.append('A unilateral edentulous area located on the right side of the mandibular arch. Kennedy classification: Class II')
  total_classIII_max=all_detected_classes.count('Not_Free_Max')
  if total_classIII_max==1:
          diagnosis.append('A unilateral edentulous area located in the maxillary arch. Kennedy classification: Class III')
  if total_classIII_max>1:
          diagnosis.append(f'{total_classIII_max} unilateral edentulous areas located in the maxillary arch. Kennedy classification: Class III')
  total_classIII_mand=all_detected_classes.count('Not_Free_Mand')
  if total_classIII_mand==1:
          diagnosis.append('A unilateral edentulous area located in the mandibular arch. Kennedy classification: Class III')
  if total_classIII_mand>1:
          diagnosis.append(f'{total_classIII_mand} unilateral edentulous areas located in the mandibular arch. Kennedy classification: Class III')
  if 'Not_Free_Center_Max' in all_detected_classes:
          diagnosis.append('A single but bilateral (crossing the midline) edentulous area located in the maxillary arch. Kennedy classification: Class IV')
  if 'Not_Free_Center_Mand' in all_detected_classes:
          diagnosis.append('A single but bilateral (crossing the midline) edentulous area located in the mandibular arch. Kennedy classification: Class IV')

  return diagnosis

# Saving Inferences and Diagnosis on Train, Valid and Test sets

In [ ]:
folder_name=['train','valid','test']
train_labels=[]
valid_labels=[]
test_labels=[]

for fn in folder_name:
  selected_folder=f'{dest_folder}/{fn}'
  selected_filenames=load_filenames(f'{selected_folder}/images')

  # Writing results to a csv file
  total_images=len(selected_filenames)
  f=open(f'{model_path}/{fn}_detailed.txt','w')

  for img_no in range(total_images):
    # Making inference
    predicted = model_train.predict(source=f'{selected_folder}/images/{selected_filenames[img_no]}',imgsz=image_size, rect=True) #here default conf is 0.25

    f.write(f'Image no.: {img_no+1}\n')
    f.write(f'Image title: {selected_filenames[img_no]}\n\n' )
    f.write('Predicted Results:\n')
    objects_predicted=len(predicted[0])
    f.write(f'Total objects detected:{objects_predicted}\n')
    f.write('Class     Class name               Bounding box coordinates (x,y,w,h)            Confidence level\n')
    all_detected_classes=[]

    for i in range(objects_predicted):
      label=int(predicted[0].boxes.data[i][5])
      #saving labels
      if fn=='train':
        train_labels.append(label)
      elif fn=='valid':
        valid_labels.append(label)
      elif fn=='test':
        test_labels.append(label)

      f.write(f'{int(predicted[0].boxes.data[i][5]):2}        ')
      all_detected_classes.append(f'{class_name[label]}')
      f.write(f'{class_name[label]:20}     ')
      w=predicted[0].boxes.xywhn[i][0].tolist()
      h=predicted[0].boxes.xywhn[i][1].tolist()
      x=predicted[0].boxes.xywhn[i][2].tolist()
      y=predicted[0].boxes.xywhn[i][3].tolist()
      f.write(f'{round(w,2):5}, {round(h,2):5}, {round(x,2):5}, {round(y,2):5}         ')
      f.write(f'{round(float(predicted[0].boxes.conf[i]),2)}\n')

    results=diagnosis(all_detected_classes)
    f.write('\nDiagnosis:\n')
    [f.write(f'{i}\n') for i in results]

    f.write('\nGround Truth\n')

    with open(f'{selected_folder}/labels/{selected_filenames[img_no][:-3]}txt') as f2:
      f.write('Class     Class name               Bounding box coordinates (x,y,w,h)\n')
      for line in f2:
        i=line.strip().split()
        f.write(f'{i[0]:2}        ')
        f.write(f'{class_name[int(i[0])]}              ')
        f.write(f'{round(float(i[1]),2)},{round(float(i[2]),2)},{round(float(i[3]),2)},{round(float(i[4]),2)}\n')
      f.write('\n')
    f.write(f'{100*"*"}\n\n')
  f.close()


WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1 /content/drive/MyDrive/AI_Dentistry/Dataset_all_YOLO/train/images/129.png: 1376x2560 1 Not_Free_Mand, 484.7ms
Speed: 30.5ms preprocess, 484.7ms inference, 1068.9ms postprocess per image at shape (1, 3, 1376, 2560)

image 1/1 /content/drive/MyDrive/AI_Dentistry/Dataset_all_YOLO/train/images/126.png: 1376x2560 1 Broken_Root, 2 Not_Free_Maxs, 1 Free_R_Mand, 1 Not_Free_Mand, 484.5ms
Speed: 21.5ms preprocess, 484.5ms inference, 3.7ms postprocess per image at shape (1, 3, 1376, 2560)

image 1/1 /content/drive/MyDrive/AI_Dentistry/Dataset_all_YOLO/train/images/127.png: 1376x2560 1 Not_Free_Max, 1 Not_Free_Center_Max, 483.1ms
Speed: 29.9ms preprocess, 483.1ms inference, 2.7ms postprocess per image at shape (1, 3, 1376, 2560)

image 1/1 /content/drive/MyDrive/AI_Dentistry/Dataset_all_YOLO/train/images/130.png: 1376x2560 1 Broken_Root, 1 Not_Free_Mand, 484.7ms
Speed: 33.3ms preprocess, 484.7ms inference, 3.6ms postprocess per image at shape (1

# Model Evaluation

In [ ]:
# Renaming 'valid' folder to 'temp'
shutil.move(f'{dest_folder}/valid',f'{dest_folder}/temp')

# Evaluating train set
shutil.move(f'{dest_folder}/train',f'{dest_folder}/valid')
metrics_train = model_train.val(data=f'{dest_folder}/data.yaml',batch=1)  # no arguments needed, dataset and settings remembered
shutil.move(f'{dest_folder}/valid',f'{dest_folder}/train')

# Evaluating test set
shutil.move(f'{dest_folder}/test',f'{dest_folder}/valid')
metrics_test = model_train.val(data=f'{dest_folder}/data.yaml',batch=1)  # no arguments needed, dataset and settings remembered
shutil.move(f'{dest_folder}/valid',f'{dest_folder}/test')

# Renaming 'temp' folder back to 'valid'
shutil.move(f'{dest_folder}/temp',f'{dest_folder}/valid')

# Evaluating valid set
metrics_valid = model_train.val(data=f'{dest_folder}/data.yaml',batch=1)  # no arguments needed, dataset and settings remembered

Ultralytics YOLOv8.0.145 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
val: Scanning /content/drive/MyDrive/AI_Dentistry/Dataset_all_YOLO/valid/labels.cache... 498 images, 105 backgrounds, 0 corrupt: 100%|██████████| 498/498 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 498/498 [06:07<00:00,  1.35it/s]
                   all        498       1733      0.788      0.599      0.701      0.519
           Broken_Root        498        492      0.837      0.445      0.628       0.42
                   PCT        498        231      0.932      0.238        0.6      0.406
            Free_R_Max        498         54      0.636      0.537      0.584      0.419
            Free_L_Max        498         66      0.831      0.621      0.699       0.52
          Not_Free_Max        498        290      0.929      0.901      0.942      0.768
   Not_Free_Center_Max        498         42      0.628      0.667 

In [ ]:
# gt_labels_train, gt_labels_valid and gt_labels_test are ground truth number of instances.
# train_labels, valid_labels and test_labels are predicted number of instances.

# Saving results for train set
train_images=load_filenames(f'{dest_folder}/train/images')
f=open(f'{model_path}/train_summary{conf}.csv','w')
f.write(f'Class,Images,Ground_Truth_Instances,Predicted_Instances,P,R,mAP50,mAP50_95\n')
f.write(f'all,{len(train_images)},{len(gt_labels_train)},{len(train_labels)},{round(metrics_train.box.mp,3)},{round(metrics_train.box.mr,3)},{round(metrics_train.box.map50,3)},{round(metrics_train.box.map,3)}\n')
c=0
for i in range(10):
  res=[-1,-1,-1,-1]
  if gt_labels_train.count(str(i))!=0:
    res=metrics_train.class_result(c)
    c+=1
  f.write(f'{i},{len(train_images)},{gt_labels_train.count(str(i))},{train_labels.count(i)},{round(res[0],3)},{round(res[1],3)},{round(res[2],3)},{round(res[3],3)}\n')
f.close()

# Saving results for valid set
valid_images=load_filenames(f'{dest_folder}/valid/images')
f=open(f'{model_path}/valid_summary{conf}.csv','w')
f.write(f'Class,Images,Ground_Truth_Instances,Predicted_Instances,P,R,mAP50,mAP50_95\n')
f.write(f'all,{len(valid_images)},{len(gt_labels_valid)},{len(valid_labels)},{round(metrics_valid.box.mp,3)},{round(metrics_valid.box.mr,3)},{round(metrics_valid.box.map50,3)},{round(metrics_valid.box.map,3)}\n')
c=0
for i in range(10):
  res=[-1,-1,-1,-1]
  if gt_labels_valid.count(str(i))!=0:
    res=metrics_valid.class_result(c)
    c+=1
  f.write(f'{i},{len(valid_images)},{gt_labels_valid.count(str(i))},{valid_labels.count(i)},{round(res[0],3)},{round(res[1],3)},{round(res[2],3)},{round(res[3],3)}\n')
f.close()

# Saving results for test set
test_images=load_filenames(f'{dest_folder}/test/images')
f=open(f'{model_path}/test_summary{conf}.csv','w')
f.write(f'Class,Images,Ground_Truth_Instances,Predicted_Instances,P,R,mAP50,mAP50_95\n')
f.write(f'all,{len(test_images)},{len(gt_labels_test)},{len(test_labels)},{round(metrics_test.box.mp,3)},{round(metrics_test.box.mr,3)},{round(metrics_test.box.map50,3)},{round(metrics_test.box.map,3)}\n')
c=0
for i in range(10):
  res=[-1,-1,-1,-1]
  if gt_labels_test.count(str(i))!=0:
    res=metrics_test.class_result(c)
    c+=1
  f.write(f'{i},{len(test_images)},{gt_labels_test.count(str(i))},{test_labels.count(i)},{round(res[0],3)},{round(res[1],3)},{round(res[2],3)},{round(res[3],3)}\n')
f.close()

# Making Solo Inference and Diagnosis

In [ ]:
folder_name='train' #train, valid or test
fn=load_filenames(f'{dest_folder}/{folder_name}/images')
c=1
for _ in fn:
  print(c,'  ',_)
  c+=1

1    129.png
2    126.png
3    127.png
4    130.png
5    132.png
6    13.png
7    136.png
8    134.png
9    135.png
10    10.png
11    14.png
12    101.png
13    102.png
14    104.png
15    103.png
16    106.png
17    105.png
18    107.png
19    109.png
20    108.png
21    111.png
22    110.png
23    115.png
24    116.png
25    118.png
26    120.png
27    123.png
28    138.png
29    137.png
30    141.png
31    140.png
32    149.png
33    148.png
34    15.png
35    144.png
36    146.png
37    145.png
38    150.png
39    153.png
40    151.png
41    152.png
42    157.png
43    156.png
44    154.png
45    161.png
46    162.png
47    163.png
48    159.png
49    16.png
50    158.png
51    167.png
52    171.png
53    17.png
54    169.png
55    165.png
56    168.png
57    173.png
58    174.png
59    176.png
60    172.png
61    175.png
62    177.png
63    178.png
64    182.png
65    181.png
66    179.png
67    185.png
68    183.png
69    184.png
70    189.png
71    186.png
72    187.png
73    1

In [ ]:
folder_name='train' #train, valid or test

img_no=1
selected_folder=f'{dest_folder}/{folder_name}'
selected_filenames=load_filenames(f'{selected_folder}/images')

In [ ]:
predicted = model_train.predict(source=f'{selected_folder}/images/{selected_filenames[img_no-1]}',imgsz=image_size)
all_detected_classes=[]

print(f'\nImage title: {selected_filenames[img_no-1]}' )

print('\nPredicted Results:')
objects_predicted=len(predicted[0])

print('Total objects detected:',objects_predicted)
print('Class\tClass name\t\tBounding box coordinates (x,y,w,h)\t   Confidence level')
for i in range(objects_predicted):
  print(f'{int(predicted[0].boxes.data[i][5]):2}', end='\t')
  all_detected_classes.append(f'{class_name[int(predicted[0].boxes.data[i][5])]}')
  print(f'{class_name[int(predicted[0].boxes.data[i][5])]:20}', end='\t')
  w=predicted[0].boxes.xywhn[i][0].tolist()
  h=predicted[0].boxes.xywhn[i][1].tolist()
  x=predicted[0].boxes.xywhn[i][2].tolist()
  y=predicted[0].boxes.xywhn[i][3].tolist()
  print(f'{round(w,2):5}, {round(h,2):5}, {round(x,2):5}, {round(y,2):5}',end='\t\t')
  print('  ',round(float(predicted[0].boxes.conf[i]),2))

results=diagnosis(all_detected_classes)
print('\nDiagnosis:')
[print(i) for i in results]

print('\nGround Truth:')
with open(f'{selected_folder}/labels/{selected_filenames[img_no-1][:-3]}txt') as f:
  print('Class\tClass name\t\tBounding box coordinates (x,y,w,h)')
  for line in f:
    i=line.strip().split()
    print(f'{i[0]:2}',end='\t')
    print(class_name[int(i[0])],end='\t\t')
    print(round(float(i[1]),2),',',round(float(i[2]),2),',',round(float(i[3]),2),',',round(float(i[4]),2))

img_ann=Image.fromarray(predicted[0].plot()[...,::-1])
img_ann

Output hidden; open in https://colab.research.google.com to view.